# Training and forecast of ML models

In [ ]:
import pandas as pd

In [ ]:
ts60 = pd.read_csv('../VEOLIA/artifacts/timeseries_60min.csv', index_col=0, parse_dates=True)
# load60 = ts60['Diff Load Activa Total (60 minuto)'].dropna()
# ts60.head()
weather = pd.read_csv('../VEOLIA/data/Burgos_weather.csv')
df = ts60[['Diff Load Activa Total (60 minuto)','TEMPERATURA EXTERIOR (60 minuto)']]
# df.head()

df.head()

In [ ]:
weather.head(1)

In [ ]:
weather['datetime'] = weather['dt_iso'].str[0:20]
weather = weather.set_index('datetime')
weather.index = pd.to_datetime(weather.index)

In [ ]:
new_df = pd.merge(df, weather, on = "datetime", how = "inner")[['Diff Load Activa Total (60 minuto)', 'temp', 'humidity', 'wind_speed', 'pressure']].dropna()

new_df.head()
df=new_df

In [ ]:
df['year'] = df.index.year
df['month'] = df.index.month
df['month_day'] = df.index.day
df['week_day'] = df.index.weekday
df['hour'] = df.index.hour


In [ ]:
df.head(150)

In [ ]:
df.shape

## Regression models for load forecasting

In [ ]:
X = df[['month', 'week_day', 'hour', 'temp', 'wind_speed','pressure', 'humidity']]
y = df['Diff Load Activa Total (60 minuto)']

In [ ]:
from sklearn.model_selection import train_test_split

# random_state=23 in order to be led to reproducible results
# split 75%, 25%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=23)
print(X_train)
print(y_train)
print(X_test)
print(y_test)

### Decision Tree Regression

In [ ]:
from sklearn import tree

tree_reg = tree.DecisionTreeRegressor()
tree_reg = tree_reg.fit(X_train, y_train)

y_train_pred_dec_tree = tree_reg.predict(X_train)
y_test_pred_dec_tree = tree_reg.predict(X_test)


In [ ]:
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error



In [ ]:
MSE_train3 = mean_squared_error(y_train, y_train_pred_dec_tree)
MSE_test3 = mean_squared_error(y_test, y_test_pred_dec_tree)

MAPE_train3 = mean_absolute_percentage_error(y_train, y_train_pred_dec_tree)
MAPE_test3 = mean_absolute_percentage_error(y_test, y_test_pred_dec_tree)


print(MSE_test3, round(MAPE_test3 * 100, 2),'%')

### Use as test set the last 11 days to compare to timeseries models

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
y_train_comp = y[:-(11*24)] 
X_train_comp = X[:-(11*24)] 


y_test_comp = y[-(11*24):] 
X_test_comp = X[-(11*24):] 
# X_train_comp = X[11*24:] 
# y_test_comp = y[:11*24]
# X_test_comp = X[:11*24]
# print(y_test_comp.shape, y_train_comp.shape, X_test_comp.shape, X_train_comp.shape)

In [ ]:
y_test_comp[10:]

In [ ]:
tree_reg = tree.DecisionTreeRegressor()
tree_reg = tree_reg.fit(X_train_comp, y_train_comp)

y_train_comp_pred_dec_tree = tree_reg.predict(X_train_comp)
y_test_comp_pred_dec_tree = tree_reg.predict(X_test_comp)

In [ ]:
MSE_train3_comp = mean_squared_error(y_train_comp, y_train_comp_pred_dec_tree)
MSE_test3_comp = mean_squared_error(y_test_comp, y_test_comp_pred_dec_tree)

MAPE_train3_comp = mean_absolute_percentage_error(y_train_comp, y_train_comp_pred_dec_tree)
MAPE_test3_comp = mean_absolute_percentage_error(y_test_comp, y_test_comp_pred_dec_tree)


print(MSE_test3_comp, round(MAPE_test3_comp * 100, 2),'%')

In [ ]:
from sklearn.metrics import mean_squared_error as mse 
from sklearn.metrics import mean_absolute_percentage_error as mape
import numpy as np
import matplotlib.pyplot as plt


ground_truth_line = pd.concat([y_train_comp[-5*24:], y_test_comp])

naive_pred = [y_train_comp.tolist()[-1]] + y_test_comp.tolist()[:-1]
print("MAPE naive:", mape(y_test_comp, naive_pred))
print("MAPE:", mape(y_test_comp, y_test_comp_pred_dec_tree))
print("MSE:", mse(y_test_comp, y_test_comp_pred_dec_tree))
print("RMSE:", np.sqrt(mse(y_test_comp, y_test_comp_pred_dec_tree)))

plt.figure()
plot = ground_truth_line.plot(figsize=(15, 3), label='Data', legend=True, title="11 day ahead forecast")
a = pd.Series(y_test_comp_pred_dec_tree)
a.index = pd.Series(y_test_comp).index
a.plot(label='Forecast', legend=True)
plot.grid()
plt.show()


### Compare to timeseries models for the last 11 days using the initially trained model

In [ ]:
from sklearn.metrics import mean_squared_error as mse 
from sklearn.metrics import mean_absolute_percentage_error as mape
import numpy as np
import matplotlib.pyplot as plt


ground_truth_line = pd.Series(y[-16*24:])

naive_pred = [y_train_comp.tolist()[-1]] + y_test_comp.tolist()[:-1]
print("MAPE naive:", mape(y_test_comp, naive_pred))
print("MAPE:", mape(y_test_comp, tree_reg.predict(X_test_comp)))
print("MSE:", mse(y_test_comp, tree_reg.predict(X_test_comp)))
print("RMSE:", np.sqrt(mse(y_test_comp, tree_reg.predict(X_test_comp))))

plt.figure()
plot = ground_truth_line.plot(figsize=(15, 3), label='Data', legend=True, title="11 day ahead forecast")
a = pd.Series(tree_reg.predict(X_test_comp))
a.index = pd.Series(y_test_comp).index
a.plot(label='Forecast', legend=True)
plot.grid()
plt.show()


In [ ]:
ground_truth_line = pd.Series(y)

# print("MAPE:", mape(y_test_comp, tree_reg.predict(X_test_comp)))
# print("MSE:", mse(y_test_comp, tree_reg.predict(X_test_comp)))
# print("RMSE:", np.sqrt(mse(y_test_comp, tree_reg.predict(X_test_comp))))

plt.figure()
plot = ground_truth_line.plot(figsize=(15, 3), label='Data', legend=True, title="Test set Decision Tree Regression forecasts")
a = pd.Series(tree_reg.predict(X_test))
a.index = pd.Series(y_test).index
a.plot(label='Forecast', legend=True)
plot.grid()
plt.show()


##### Hyperparameter tuning

In [ ]:

parameters={"splitter":["best","random"],
            "max_depth" : [1,2,5,10],
           "min_samples_leaf":[2,5,8,10],
           "min_weight_fraction_leaf":[0.1,0.2,0.5],
           "max_features":["auto","log2","sqrt",None],
           "max_leaf_nodes":[None,10,20,50,90] }

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
tuning_model=GridSearchCV(tree_reg,param_grid=parameters,scoring='neg_mean_squared_error',cv=4,verbose=3)
tuning_model.fit(X,y)

In [ ]:
tuning_model.best_params_

In [ ]:
# best model score
tuning_model.best_score_

In [ ]:
best_dec_tree_pred = tuning_model.best_estimator_.predict(X_test)

In [ ]:
MSE_test_bdt = mean_squared_error(y_test, best_dec_tree_pred)

MAPE_test_bdt = mean_absolute_percentage_error(y_test, best_dec_tree_pred)
print(MSE_test_bdt, round(MAPE_test_bdt * 100, 2),'%')

### Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor()
regr.fit(X_train, y_train)

y_train_pred_rf = regr.predict(X_train)
y_test_pred_rf = regr.predict(X_test)

In [ ]:
MSE_train_rf = mean_squared_error(y_train, y_train_pred_rf)
MSE_test_rf = mean_squared_error(y_test, y_test_pred_rf)

MAPE_train_rf = mean_absolute_percentage_error(y_train, y_train_pred_rf)
MAPE_test_rf = mean_absolute_percentage_error(y_test, y_test_pred_rf)


print(MSE_test_rf, round(MAPE_test_rf * 100, 2),'%')

In [ ]:
ground_truth_line = pd.Series(y)

# print("MAPE:", mape(y_test_comp, tree_reg.predict(X_test_comp)))
# print("MSE:", mse(y_test_comp, tree_reg.predict(X_test_comp)))
# print("RMSE:", np.sqrt(mse(y_test_comp, tree_reg.predict(X_test_comp))))

plt.figure()
plot = ground_truth_line.plot(figsize=(15, 3), label='Data', legend=True, title="Test set Random Forest Regression forecasts")
a = pd.Series(regr.predict(X_test))
a.index = pd.Series(y_test).index
a.plot(label='Forecast', legend=True)
plot.grid()
plt.show()

In [ ]:
parameters={"max_depth" : [1,2,5,10],
           "min_samples_split":[1,2,5,10],
           "min_samples_leaf":[1,2,5,10],
            "n_estimators":[10, 50, 100, 200],
           "min_weight_fraction_leaf":[0.1,0.2,0.5],
           "max_leaf_nodes":[10,50,90] }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

tuning_model=RandomizedSearchCV(estimator=regr, param_distributions=parameters, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
tuning_model.fit(X,y)

In [ ]:
tuning_model.best_params_

In [ ]:
tuning_model.best_score_

In [ ]:
best_rf_pred = tuning_model.best_estimator_.predict(X_test)

In [ ]:
MSE_test_brf = mean_squared_error(y_test, best_rf_pred)

MAPE_test_brf = mean_absolute_percentage_error(y_test, best_rf_pred)
print(MSE_test_brf, round(MAPE_test_brf * 100, 2),'%')

### Gradient Boosting Regression

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_train, y_train)

y_train_pred_gb = reg.predict(X_train)
y_test_pred_gb = reg.predict(X_test)

In [ ]:
MSE_train_gb = mean_squared_error(y_train, y_train_pred_gb)
MSE_test_gb = mean_squared_error(y_test, y_test_pred_gb)

MAPE_train_gb = mean_absolute_percentage_error(y_train, y_train_pred_gb)
MAPE_test_gb = mean_absolute_percentage_error(y_test, y_test_pred_gb)


print(MSE_test_gb, round(MAPE_test_gb * 100, 2),'%')

In [ ]:
ground_truth_line = pd.Series(y)

# print("MAPE:", mape(y_test_comp, tree_reg.predict(X_test_comp)))
# print("MSE:", mse(y_test_comp, tree_reg.predict(X_test_comp)))
# print("RMSE:", np.sqrt(mse(y_test_comp, tree_reg.predict(X_test_comp))))

plt.figure()
plot = ground_truth_line.plot(figsize=(15, 3), label='Data', legend=True, title="Test set Gradient Boosting Regression forecasts")
a = pd.Series(reg.predict(X_test))
a.index = pd.Series(y_test).index
a.plot(label='Forecast', legend=True)
plot.grid()
plt.show()

### LightGBM

In [ ]:
import lightgbm as lgb

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}



In [ ]:

# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_test,
                early_stopping_rounds=5)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')


In [ ]:
# predict
lgbm_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
mape_test = mean_absolute_percentage_error(y_test, lgbm_pred)
print(f'The MAPE of prediction is: {mape_test}')

In [ ]:
ground_truth_line = pd.Series(y)

# print("MAPE:", mape(y_test_comp, tree_reg.predict(X_test_comp)))
# print("MSE:", mse(y_test_comp, tree_reg.predict(X_test_comp)))
# print("RMSE:", np.sqrt(mse(y_test_comp, tree_reg.predict(X_test_comp))))

plt.figure()
plot = ground_truth_line.plot(figsize=(15, 3), label='Data', legend=True, title="Test set LightGBM Regression forecasts")
a = pd.Series(gbm.predict(X_test, num_iteration=gbm.best_iteration))
a.index = pd.Series(y_test).index
a.plot(label='Forecast', legend=True)
plot.grid()
plt.show()

### XGBoost

In [ ]:
import xgboost

xgb = xgboost.XGBRegressor()
xgb.fit(X_train, y_train)

In [ ]:
xgb_pred = xgb.predict(X_test)

xgb_mape = mean_absolute_percentage_error(y_test, xgb_pred)
print(f'The MAPE of prediction is: {xgb_mape}')

In [ ]:
ground_truth_line = pd.Series(y)

# print("MAPE:", mape(y_test_comp, tree_reg.predict(X_test_comp)))
# print("MSE:", mse(y_test_comp, tree_reg.predict(X_test_comp)))
# print("RMSE:", np.sqrt(mse(y_test_comp, tree_reg.predict(X_test_comp))))

plt.figure()
plot = ground_truth_line.plot(figsize=(15, 3), label='Data', legend=True, title="Test set XGBoost Regression forecasts")
a = pd.Series(xgb.predict(X_test))
a.index = pd.Series(y_test).index
a.plot(label='Forecast', legend=True)
plot.grid()
plt.show()

### AutoGluon

In [ ]:
!pip install autogluon

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
label_column = ''
print("Summary of class variable: \n", train_data[label_column].describe())

In [ ]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)